In [1]:
# Imports and Utility functions
import subprocess

def clear_keri():
    path = "/usr/local/var/keri/"
    confirm = input("🚨 This will clear your keystore. Are you sure? (y/n): ")
    if confirm.lower() == "y":
        print("Proceeding...")
        try:
            subprocess.run(["rm", "-rf", path], check=True)
            print(f"✅ Successfully removed: {path}")
        except subprocess.CalledProcessError as e:
            print(f"❌ Error removing {path}: {e}")
    else:
        print("Operation cancelled.")

# Signatures

Having explored KERI Identifiers (AIDs) and their management via Controllers and the KLI, we now turn to a core mechanism underpinning KERI's security and trust: digital signatures. This section will outline what signatures are, their significance, and how they function within KERI.

## What is a Digital Signature?

At its heart, a digital signature is a cryptographic technique used to verify the authenticity and integrity of digital messages or documents. Think of it like a handwritten signature in the physical world, but with much stronger guarantees provided by cryptography.

Here's the basic idea:

1.  **Signing:** The sender (or Controller in KERI) uses their unique private key along with the message content to generate a unique piece of data – the signature. This process typically involves hashing the message first and then encrypting the hash with the private key
2.  **Attaching:** This signature is attached to the original message
3.  **Verification:** Anyone who receives the message can use the sender's corresponding public key to verify the signature. The verification process checks if the signature corresponds correctly to the message content and the sender's public key

If the verification succeeds, the receiver has high confidence that:

* **Authenticity:** The message genuinely came from the owner of the private key (the purported sender)
* **Integrity:** The message has not been altered or tampered with since it was signed

## Verify a Signature

The verifying a signature refers to the process used by the receiver (the Verifier) to assess the authenticity of a message. Specifically, it involves applying an algorithm that, given the message, the public key, and the signature, determines whether to accept or reject the message's claim to authenticity. If the algorithm accepts, the signature is considered valid. If it rejects, the signature is deemed invalid — indicating that the message may have been tampered with, the signature is corrupt, or it was not generated by the holder of the corresponding private key.

## Non-Repudiation

Digital signatures provide a crucial property known as non-repudiation, which ensures that the author of a statement cannot deny its authorship or the validity of an associated contract, signature, or commitment. In simpler terms, once someone signs a message with their private key and that signature is successfully verified using their public key, they cannot credibly deny having signed it. This is because only the legitimate holder of the private key should be able to generate that specific signature. The concept of non-repudiation often arises in legal contexts where the authenticity of a signature is challenged, or "repudiated."

## Signatures and Verification in KERI

KERI uses digital signatures to establish trust. Their primary function is to ensure the authenticity of data (like key events) by verifiably linking it to a specific Identifier. This process involves signing by a Controller and verification by a Verifier.

To verify a signature, the Verifier must perform these key steps:

1.  **Find the Public Key** — The Verifier identifies the correct public key for the AID corresponding to the time the message was signed. How this key is found depends on the AID's characteristics; it might be derived directly from the AID prefix or require consulting the AID's Key Event Log (KEL) to account for potential key changes over time.
2.  **Verify the Signature** — Using the correctly identified public key(s), the Verifier runs the verification algorithm on the signature attached to the message.


<div class="alert alert-info">
  <b>ℹ️ NOTE</b><hr>
    There's a subtle difference between a Verifier (who checks cryptographic validity according to KERI rules) and a Validator (who might perform broader checks, including business logic or schema validation, in addition to verification). In KERI discussions, "Verifier" typically emphasizes the cryptographic checks.
</div>


## Practical Example: Signing and Verifying with KLI

Let's see how signing and verification work using the KLI commands.

### Setup Keystore and Identifier

First, we need a keystore and an identifier.



In [2]:
keystore_name="signature-key-store"
keystore_passcode="xSLg286d4iWiRg2mzGYca"
salt="0ABeuT2dErMrqFE5Dmrnc2Bq"
aid_alias = "aid-sign"

!kli init --name {keystore_name} --passcode {keystore_passcode} --salt {salt}
!kli incept --name {keystore_name} --alias {aid_alias} --passcode {keystore_passcode} \
    --icount 1 --isith 1 --ncount 1 --nsith 1 --toad 0

KERI Keystore created at: /usr/local/var/keri/ks/signature-key-store
KERI Database created at: /usr/local/var/keri/db/signature-key-store
KERI Credential Store created at: /usr/local/var/keri/reg/signature-key-store
	aeid: BD-1udeJaXFzKbSUFb6nhmndaLlMj-pdlNvNoN562h3z
ERR: Already incepted pre=BPkrtxL07k1cQhoRlF2illnFZJc_Veywvg4xvLFPtCE3.


### Sign a Message 

Now, let's sign a simple text message `"hello world"` using the private key associated with our `aid-sign` identifier.

In [3]:
!kli sign --name {keystore_name} --alias {aid_alias} --passcode {keystore_passcode} --text "hello world"

1. AAAzpKaOVx-4G1cfldpRx-mHi1UGS-smitfVCSM47v_mECPkgHVPfqOogqseh8GwPyua0JFi3wNPRHEnowxgDYUE


The output is the digital signature generated for the text "hello world" using the private key of the AID  

### Verify the Signature 

We can now use the kli verify command to check if the signature is valid for the given message and identifier (prefix).

In [5]:
!kli verify --name {keystore_name} --alias {aid_alias} --passcode {keystore_passcode} \
    --prefix BPkrtxL07k1cQhoRlF2illnFZJc_Veywvg4xvLFPtCE3 \
    --text "hello world" \
    --signature AAAzpKaOVx-4G1cfldpRx-mHi1UGS-smitfVCSM47v_mECPkgHVPfqOogqseh8GwPyua0JFi3wNPRHEnowxgDYUE


Signature 1 is valid.


The command confirms the signature is valid. It used the public key associated with the prefix  (which KLI retrieves from the keystore in this case) to verify the signature against the provided text.

### Tampered Signature

What happens if the signature is altered even slightly? Let's change the last character of the signature and try to verify again.


In [7]:
!kli verify --name {keystore_name} --alias {aid_alias} --passcode {keystore_passcode} \
    --prefix BPkrtxL07k1cQhoRlF2illnFZJc_Veywvg4xvLFPtCE3 \
    --text "hello world" \
    --signature AAAzpKaOVx-4G1cfldpRx-mHi1UGS-smitfVCSM47v_mECPkgHVPfqOogqseh8GwPyua0JFi3wNPRHEnowxgDYUU # Tampered last character

ERR: Signature 1 is invalid.


As expected, the verification fails. Even a tiny change invalidates the signature, demonstrating the integrity protection it provides.